In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map = geemap.Map()
Map

# 有效像元分布，2000-2020

## 计算

In [ ]:
# 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
region = 'wuhan'
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

In [ ]:
def near_orbit(imgCol):
    imgCol_date = imgCol.map(lambda i:i.set({'date': ee.String(i.get('system:id')).slice(-8,)}))
    dates = ee.List(imgCol_date.distinct('date').aggregate_array('date'))
    imgCol_median = ee.ImageCollection(dates.map(lambda i:imgCol_date.filter(ee.Filter.eq('date',i)).median().copyProperties(imgCol_date.filter(ee.Filter.eq('date',i)).first())))
    return imgCol_median

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2013-01-01', '2020-12-31') 
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('1999-01-01', '2020-12-31')        
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('1999-01-01', '2012-12-31')
print(l8.size().getInfo())
print(l7.size().getInfo())
print(l5.size().getInfo())


landsat_images =near_orbit(l8.map(maskSR)).merge(near_orbit(l7.map(maskSR))).merge(near_orbit(l5.map(maskSR)))
validPixel = ee.ImageCollection(landsat_images).count().select('B3').clip(roi).rename('count')

In [ ]:
task = ee.batch.Export.image.toAsset(**{
    'image': validPixel,
    'description': region,
    'assetId': 'users/311605001111/validPixel_' + region,
    'scale': 30,
    'region': roi,
    'maxPixels':1e13,
})
task.start()

In [ ]:
validPixel = ee.ImageCollection(landsat_images).count().select('B3').clip(roi).rename('count')
Map.addLayer(validPixel, {'palette':['white','blue'],'min':0,'max':1000},'validPixel')

## 制图

In [ ]:
## 大庆
# validPixel = ee.Image('users/311605001111/validPixel_daqing' + 'v2')
validPixel = ee.Image('users/311605001111/validPixel_daqing')
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [123.9355, 45.8362,125.3055, 46.8762] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1400}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region A', fontsize=18)

In [ ]:
## 天津
validPixel = ee.Image('users/311605001111/validPixel_tianjin' + 'v2')
roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [115.9244, 38.6272,117.4944,39.6372] 
fig = plt.figure(figsize=(10, 8))
visParams = {'min': 0.0,'max': 1400}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region B', fontsize=18)

In [ ]:
## 武汉
validPixel = ee.Image('users/311605001111/validPixel_wuhan' + 'v2')
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.7393, 29.8642,115.0993, 30.9242] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1400}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region C', fontsize=18)

In [ ]:
## 珠三角
validPixel = ee.Image('users/311605001111/validPixel_prd' + 'v2')
roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [112.7614, 22.2347,114.0514, 23.2547] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1400}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
# cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region D', fontsize=18)

In [ ]:
## 青藏高原
validPixel = ee.Image('users/311605001111/validPixel_tibet' + 'v2')
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 有效像元
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8826, 34.6579,91.3626, 35.8279] 
fig = plt.figure(figsize=(10, 6))
visParams = {'min': 0.0,'max': 1400}

ax = cartoee.get_map(validPixel,cmap="rainbow",region=region,vis_params=visParams)
cartoee.add_colorbar(ax,vis_params=visParams,cmap="rainbow",loc="right",label="number",orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'Region E', fontsize=18)